# Libraries

In [125]:
import pandas as pd
from getpass import getuser
from collections import defaultdict

# Load and inspect dataset

In [126]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\fifa.csv'

# Read the dataset with a different encoding
df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Extract relevant columns

In [127]:
# First, let's extract the year from the 'tournament_id' column and create a new column 'year'
df['year'] = df['tournament_id'].str.extract(r'WC-(\d{4})').astype(int)

# Now, filter the data for tournaments after 1986
filtered_df = df[df['year'] > 1986]

# Extract relevant columns for goal events and match results
goals_df = filtered_df[['tournament_name', 'group_name','match_name', 'match_id', 'player_team_name','match_date', 'minute_regulation', 
               'team_id', 'own_goal']]

# Remove all observations where group_name is "not applicable"
goals_df = goals_df[goals_df['group_name'] != 'not applicable']

# Convert 'match_date' to datetime format
goals_df['match_date'] = pd.to_datetime(goals_df['match_date'], format='%m/%d/%Y')

# Sort the dataset by 'match_date' in ascending order (oldest first) and 'minute_regulation'
goals_df = goals_df.sort_values(by=['match_date', 'minute_regulation'], ascending=[True, True])

# Display the first few rows to confirm the sorting
goals_df.head()


,tournament_name,group_name,match_name,match_id,player_team_name,match_date,minute_regulation,team_id,own_goal
1328,1990 FIFA World Cup,Group B,Argentina v Cameroon,M-1990-01,Cameroon,1990-06-08,67,T-11,0
1329,1990 FIFA World Cup,Group B,Soviet Union v Romania,M-1990-02,Romania,1990-06-09,41,T-59,0
1331,1990 FIFA World Cup,Group D,United Arab Emirates v Colombia,M-1990-03,Colombia,1990-06-09,50,T-15,0
1330,1990 FIFA World Cup,Group B,Soviet Union v Romania,M-1990-02,Romania,1990-06-09,55,T-59,0
1333,1990 FIFA World Cup,Group A,Italy v Austria,M-1990-04,Italy,1990-06-09,78,T-39,0


# Calculate dynamically match outcomes after each goal

### Filter out the matches from the last match day in the goals_df DataFrame.

In [128]:

# Find the last match date for each tournament and group
last_dates = goals_df.groupby(['tournament_name', 'group_name'])['match_date'].max().reset_index()

# Exclude the matches from the last match day by merging the last_dates and removing them
goals_before_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'], how='outer', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Split match_name into home and away teams and remove match_name column
goals_before_last_day[['home', 'away']] = goals_before_last_day['match_name'].str.split(' v ', expand=True)
goals_before_last_day = goals_before_last_day.drop(columns=['match_name'])

# Display the filtered dataset
goals_before_last_day.head()


,tournament_name,group_name,match_id,player_team_name,match_date,minute_regulation,team_id,own_goal,home,away
0,1990 FIFA World Cup,Group B,M-1990-01,Cameroon,1990-06-08,67,T-11,0,Argentina,Cameroon
1,1990 FIFA World Cup,Group B,M-1990-02,Romania,1990-06-09,41,T-59,0,Soviet Union,Romania
2,1990 FIFA World Cup,Group B,M-1990-02,Romania,1990-06-09,55,T-59,0,Soviet Union,Romania
3,1990 FIFA World Cup,Group D,M-1990-03,Colombia,1990-06-09,50,T-15,0,United Arab Emirates,Colombia
4,1990 FIFA World Cup,Group D,M-1990-03,Colombia,1990-06-09,85,T-15,0,United Arab Emirates,Colombia


## create df  of aggregate data for each match

In [129]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['tournament_name', 'group_name', 'match_id', 'home', 'away']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Loop through each row in the group
    for _, row in group.iterrows():
        # Check if the team is the home team and no own goal
        if row['home'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_home += 1
        # If the team is the away team and no own goal, increment goals_away
        elif row['away'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_away += 1

    # Append the results to the list, keeping home and away columns
    results.append({
        'tournament_name': match_id[0],
        'group_name': match_id[1],
        'match_id': match_id[2],
        'home': match_id[3],
        'away': match_id[4],
        'goals_home': goals_home,
        'goals_away': goals_away
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

# Display the updated DataFrame (use print if outside of Jupyter)
display(agg_goals_before_last_day)



,tournament_name,group_name,match_id,home,away,goals_home,goals_away,won
0,1990 FIFA World Cup,Group A,M-1990-04,Italy,Austria,1,0,1
1,1990 FIFA World Cup,Group A,M-1990-05,United States,Czechoslovakia,1,5,-1
2,1990 FIFA World Cup,Group A,M-1990-16,Italy,United States,1,0,1
3,1990 FIFA World Cup,Group A,M-1990-17,Austria,Czechoslovakia,0,1,-1
4,1990 FIFA World Cup,Group B,M-1990-01,Argentina,Cameroon,0,1,-1
...,...,...,...,...,...,...,...,...
244,2022 FIFA World Cup,Group G,M-2022-29,Cameroon,Serbia,3,3,0
245,2022 FIFA World Cup,Group G,M-2022-31,Brazil,Switzerland,1,0,1
246,2022 FIFA World Cup,Group H,M-2022-15,Portugal,Ghana,3,2,1
247,2022 FIFA World Cup,Group H,M-2022-30,South Korea,Ghana,2,3,-1


## aggregate data for home and away games

In [130]:
# Step 1: Home games aggregation
home_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'home']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: sum(3 if result == 1 else (1 if result == 0 else 0) for result in x))
).reset_index()

# Step 2: Away games aggregation
away_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'away']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: sum(3 if result == -1 else (1 if result == 0 else 0) for result in x))
).reset_index()


## aggregate data after first two matches

In [132]:
# Step 1: Merge home_games and away_games on tournament_name, group_name, and home with away
all_games_before_last = pd.merge(
    home_games,
    away_games,
    left_on=['tournament_name', 'group_name', 'home'],
    right_on=['tournament_name', 'group_name', 'away'],
    how='outer',
    suffixes=('_home', '_away')
)

# Step 2: For teams that only appear in home_games, assign home to team and copy relevant columns
all_games_before_last['team'] = all_games_before_last['home'].fillna(all_games_before_last['away'])

# Step 3: For goals_scored, goals_conceded, and points, handle missing values
all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

# Step 4: Add goals_difference column
all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

# Step 5: Drop unnecessary columns used in the merge process
all_games_before_last = all_games_before_last[['tournament_name', 'group_name', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference']]

# Step 6: Sort by points, goals_difference, and goals_scored within each tournament and group
all_games_before_last = all_games_before_last.sort_values(
    by=['tournament_name', 'group_name', 'points', 'goals_difference', 'goals_scored'],
    ascending=[True, True, False, False, False]
).reset_index(drop=True)

# Display the final sorted dataframe
display(all_games_before_last)


,tournament_name,group_name,team,goals_scored,goals_conceded,points,goals_difference
0,1990 FIFA World Cup,Group A,Czechoslovakia,6.0,1.0,6.0,5.0
1,1990 FIFA World Cup,Group A,Italy,2.0,0.0,6.0,2.0
2,1990 FIFA World Cup,Group A,Austria,0.0,2.0,0.0,-2.0
3,1990 FIFA World Cup,Group A,United States,1.0,6.0,0.0,-5.0
4,1990 FIFA World Cup,Group B,Cameroon,3.0,1.0,6.0,2.0
...,...,...,...,...,...,...,...
266,2022 FIFA World Cup,Group G,Serbia,3.0,5.0,1.0,-2.0
267,2022 FIFA World Cup,Group H,Portugal,5.0,2.0,6.0,3.0
268,2022 FIFA World Cup,Group H,Ghana,5.0,5.0,3.0,0.0
269,2022 FIFA World Cup,Group H,South Korea,2.0,3.0,0.0,-1.0
